Universidade Federal de Alagoas

IC - Instituto de Computação

 

# Processamento de linguagem natural - 2020.1
**Professor**: Thales Vieira

**Alunos**: Hugo Tallys Martins Oliveira e Valério Nogueira Rodrigues Júnior


## 6ª lista de exercícios

---

### Pré-processamento dos dados

In [1]:
import nltk
import numpy
import pandas
import random
import re
import time
import umap

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20
from bokeh.plotting import figure
from gensim.models import KeyedVectors
from IPython.display import HTML, display
from keras.layers import Conv1D, Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF, PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from yellowbrick.cluster import KElbowVisualizer

c:\users\valerio nogueira\python 3.8.5\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
output_notebook() # Necessário para visualizar os gráficos com bokeh

Loading BokehJS ...

In [3]:
nltk.download('stopwords'); nltk.download('rslp'); nltk.download('punkt');

[nltk_data] Downloading package stopwords to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
bbc_dataset_url = 'data/bbc.csv'
cnn_dataset_url = 'data/cnn.csv'

bbc = pandas.read_csv(bbc_dataset_url, sep='|')
bbc['label'] = 'BBC'
cnn = pandas.read_csv(cnn_dataset_url, sep='|')
cnn['label'] = 'CNN'

dataset = pandas.concat([bbc, cnn], ignore_index=True)
dataset = dataset.dropna(axis=0).reset_index(drop=True)

In [5]:
dataset.text = dataset.text.apply(lambda text: text.replace('\n', ' ')) # Remoção das quebras de linha
dataset.title = dataset.title.apply(lambda text: text.replace('\n', ' '))

In [6]:
def remove_boilerplate(text):
    boilerplate = ['Compartilhe este post com Email Facebook Messenger Messenger Twitter WhatsApp LinkedIn Copiar este link Estes são links externos e abrirão numa nova janela', 'Já assistiu aos nossos novos vídeos no YouTube? Inscreva-se no nosso canal!', 'Final de YouTube post  de BBC News Brasil Final de YouTube post 2 de BBC News Brasil Final de YouTube post 3 de BBC News Brasil']
    
    for b in boilerplate:
        text = text.replace(b, '')
    return text

dataset.text = dataset.text.apply(remove_boilerplate) # Remoção de fragmentos irrelevantes do texto que se repetem

In [7]:
def preprocess(text):
    text = re.sub(r'\w*\d\w*', '', text) # Remove todas as palavras que contém números
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', text.lower()) # Remove pontuação e converte para minúscula
    return re.sub(r'\s+', ' ', text) # Remove espaços repetidos

dataset['processed_text'] = dataset.text.apply(preprocess)
dataset['processed_title'] = dataset.title.apply(preprocess)

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def tokenize_remove_stopwords(text):
    tokenized_text = nltk.word_tokenize(text, language='portuguese')
    return " ".join([token for token in tokenized_text if token not in stopwords])

dataset.processed_text = dataset.processed_text.apply(tokenize_remove_stopwords) # Tokeniza o texto e remove stopwords
dataset.processed_title = dataset.processed_title.apply(tokenize_remove_stopwords) # Tokeniza o titulo e remove stopwords

In [9]:
dataset

,url,title,text,label,processed_text,processed_title
0,https://www.bbc.co.uk/portuguese/brasil-53020785,Coronavírus: pandemia pode jogar até 14 milhõe...,A turbulência econômica causada pela pandemi...,BBC,turbulência econômica causada pandemia novo co...,coronavírus pandemia pode jogar milhões brasil...
1,https://www.bbc.co.uk/portuguese/brasil-53027318,Coronavírus: como funcionam as duas vacinas co...,Cerca de 11 mil voluntários brasileiros vão ...,BBC,cerca mil voluntários brasileiros vão receber ...,coronavírus funcionam duas vacinas contra covi...
2,https://www.bbc.co.uk/portuguese/brasil-51713943,Coronavírus: Brasil passa o Reino Unido e se t...,*atualizada às 18h20 de 12 de junho de 2020 ...,BBC,atualizada s junho brasil totalizou nesta sext...,coronavírus brasil passa reino unido torna seg...
3,https://www.bbc.co.uk/portuguese/internacional...,Coronavírus na Índia: com lockdown 'insustentá...,"Quando, em 24 de março, o governo indiano in...",BBC,março governo indiano iniciou estrito isolamen...,coronavírus índia lockdown insustentável índia...
4,https://www.bbc.co.uk/portuguese/internacional...,2ª onda do coronavírus? Irã vê aumento acelera...,O Irã registrou um rápido aumento no número ...,BBC,irã registrou rápido aumento número casos covi...,onda coronavírus irã vê aumento acelerado após...
...,...,...,...,...,...,...
1611,https://www.cnnbrasil.com.br/saude/2020/02/27/...,Farmácias têm falta de máscaras após confirmaç...,Com a confirmação do primeiro caso de contamin...,CNN,confirmação primeiro caso contaminação novo co...,farmácias têm falta máscaras após confirmação ...
1612,https://www.cnnbrasil.com.br/business/2020/02/...,Ibovespa tem nova queda com mercado ainda preo...,Preocupações com a propagação do novo coronaví...,CNN,preocupações propagação novo coronavírus poten...,ibovespa nova queda mercado ainda preocupado c...
1613,https://www.cnnbrasil.com.br/internacional/202...,Japonesa testa positivo pela segunda vez para ...,TÓQUIO - Uma guia de ônibus turístico no Japão...,CNN,tóquio guia ônibus turístico japão apresentou ...,japonesa testa positivo segunda vez coronavírus
1614,https://www.cnnbrasil.com.br/saude/2020/02/26/...,Primeiro brasileiro com coronavírus tem sintom...,O primeiro brasileiro com diagnóstico confir...,CNN,primeiro brasileiro diagnóstico confirmado cor...,primeiro brasileiro coronavírus sintomas brand...


In [10]:
def get_vocab(texts):
    vocab = ' '.join([text for text in texts])
    return sorted(set(vocab.split(' ')))

processed_texts_vocab = get_vocab(dataset.processed_text.values)
processed_titles_vocab = get_vocab(dataset.processed_title.values)

#### Tokenizer

In [199]:
TOKENIZER_NUM_WORDS = 2000

tokenizer = Tokenizer(num_words=TOKENIZER_NUM_WORDS)
tokenizer.fit_on_texts(dataset.processed_text)

### Embedding, convoluções e LSTM

Resolva novamente a segunda questão da 3ª lista usando pelo menos duas
arquiteturas de redes neurais que utilizem camadas *Embedding*, convolucionais e
*LSTM*. Compare com os resultados obtidos anteriormente nas lista 3 e 5.

#### Gerando as sequências

In [12]:
sequences = tokenizer.texts_to_sequences(dataset.processed_text)
sequences = pad_sequences(sequences)

In [25]:
INPUT_DIMENSION = TOKENIZER_NUM_WORDS + 1
INPUT_LENGTH = 500
CONV_FILTERS = 32
CONV_KERNEL_SIZE = 3
LSTM_UNITS = 100
DENSE_UNITS = 16

In [26]:
lstm_model = Sequential([
    Embedding(input_dim=INPUT_DIMENSION, output_dim=300),
    Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL_SIZE),
    LSTM(units=LSTM_UNITS),
    Dense(units=DENSE_UNITS),
    Dense(units=1, activation='sigmoid')
])

print(lstm_model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         1800      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          28832     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1616      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 85,465
Trainable params: 85,465
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.25

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(x=sequences, 
               y=dataset.label.apply(lambda label: 0 if label == 'BBC' else 1),
               batch_size=BATCH_SIZE,
               validation_split=VALIDATION_SPLIT,
               epochs=20)

# Geração de Texto

2. Usando sua base de textos:

* Treine uma rede __LSTM__ para gerar texto, que receba uma ou mais palavras de uma frase como entrada. O treinamento deve ser realizado considerando um conjunto supervisionado que gera a próxima palavra de uma sequência de tamanho 4, usando subsequências de sua base.

* Após o treinamento, exiba pelo menos 5 exemplos de textos dados de entrada, e do texto gerado em seguida pela rede treinada. Para cada exemplo, gere pelo menos 10 palavras consecutivamente.

* Faça o mesmo usando Cadeias de Markov com bi-grams (usando apenas 1 palavra para tentar prever a seguinte). Compare os resultados com os da __LSTM__.



In [200]:
def get_one_hot_sequence(s):
    _ = []
    for ts in s:
        _.append(list(map(lambda t: tokens_one_hot[t], ts)))
    return numpy.array(_)

In [201]:
sequences = tokenizer.texts_to_sequences(dataset.processed_text[:50])
sequences = pad_sequences(sequences, padding='post')

tokens = list(tokenizer.word_index.values())[:TOKENIZER_NUM_WORDS]

cat_tokens = to_categorical(tokens)

tokens_one_hot = {
    t: cat_tokens[i] for i, t in enumerate(tokens)
}
tokens_one_hot[0] = numpy.zeros(shape=TOKENIZER_NUM_WORDS + 1)

def generate_subsequences(sequence):
  seq, tar = [], []
  win = [0, 0, 0, sequence[0]]

  for i in sequence[1:]:
    seq.append(win.copy()); tar.append(i)
    for j in range(1, 4):
      win[j-1] = win[j]
    win[3] = i
  
  return seq, tar

train_sequences, train_targets = [], []

for s in sequences:
    seq, tar = generate_subsequences(s)
    train_sequences.append(seq)
    train_targets.append(tar)

train_sequences = get_one_hot_sequence(numpy.array([s for seq in train_sequences for s in seq]))
train_targets = numpy.array([tokens_one_hot[t] for tar in train_targets for t in tar])

In [202]:
INPUT_DIMENSION = TOKENIZER_NUM_WORDS + 1
EMBEDDING_OUTPUT_DIM = 300
INPUT_LENGTH = 4
LSTM_UNITS = 100

lstm_model = Sequential([
    LSTM(units=LSTM_UNITS, input_shape=(INPUT_LENGTH, INPUT_DIMENSION)),
    Dense(units=INPUT_DIMENSION, activation='softmax')
])

print(lstm_model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 100)               840800    
_________________________________________________________________
dense_16 (Dense)             (None, 2001)              202101    
Total params: 1,042,901
Trainable params: 1,042,901
Non-trainable params: 0
_________________________________________________________________
None


In [203]:
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.25

lstm_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [204]:
lstm_model.fit(
    x=train_sequences, 
    y=train_targets,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    epochs=20, verbose=1
)

Epoch 1/20
172/172 [==============================] - 29s 169ms/step - loss: 2.5084 - accuracy: 0.0087 - val_loss: 2.7041 - val_accuracy: 0.0047
Epoch 2/20
172/172 [==============================] - 17s 98ms/step - loss: 2.4147 - accuracy: 0.0052 - val_loss: 2.7061 - val_accuracy: 0.0047
Epoch 3/20
172/172 [==============================] - 16s 92ms/step - loss: 2.4117 - accuracy: 0.0052 - val_loss: 2.7148 - val_accuracy: 0.0047
Epoch 4/20
172/172 [==============================] - 17s 97ms/step - loss: 2.4112 - accuracy: 0.0052 - val_loss: 2.7133 - val_accuracy: 0.0047
Epoch 5/20
172/172 [==============================] - 16s 93ms/step - loss: 2.4105 - accuracy: 0.0052 - val_loss: 2.7171 - val_accuracy: 0.0047
Epoch 6/20
172/172 [==============================] - 16s 92ms/step - loss: 2.4104 - accuracy: 0.0049 - val_loss: 2.7175 - val_accuracy: 0.0047
Epoch 7/20
172/172 [==============================] - 16s 91ms/step - loss: 2.4092 - accuracy: 0.0050 - val_loss: 2.7194 - val_accuracy

In [ ]:
def gen_text(initial, max_len=500):
    initial = get_one_hot_sequence(pad_sequences(tokenizer.texts_to_sequences([initial])[:4], 4, padding='post'))
    text = []
    
    print(initial)
    
    for _ in range(max_len):
        predict = numpy.argmax(lstm_model.predict([initial]))
        initial[0][:-1] = initial[0][1:]; initial[0][-1] = tokens_one_hot[predict]
        text.append(tokenizer.index_word[predict])
    
    return ' '.join(text)

gen_text(input('Palavras inicias'), max_len=int(input('Tamanho do texto')))
# O coronavírus no Brasil
